## crawling

In [ ]:
import sys, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from pandas import DataFrame
from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm

In [ ]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5

In [ ]:
options = webdriver.ChromeOptions()
browser = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver", options=options)
browser.execute_script('return navigator.userAgent')
browser.close()

In [ ]:
news_dict = {}
idx = 0
page = 1
isBreak = False

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36")
options.add_argument("--blink-settings=imagesEnabled=false")

browser = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")

# journal, date, title, url, content
while(1):
    browser.get('https://www.donga.com/news/List?p={}&prod=news&ymd=&m=NP'.format(page))
    browser.implicitly_wait(sleep_sec)

    for i in tqdm(range(2, 22)):
        try:
            news_date = browser.find_element(by=By.XPATH, value='//*[@id="content"]/div[{}]/div[@class="rightList"]/a/span[@class="date"]'.format(i)).text
            news_title = browser.find_element(by=By.XPATH, value='//*[@id="content"]/div[{}]/div[@class="rightList"]/a/span[@class="tit"]'.format(i)).text
            news_url = browser.find_element(by=By.XPATH, value='//*[@id="content"]/div[{}]/div[@class="rightList"]/a'.format(i)).get_attribute('href')

            try:
                date = datetime.fromisoformat(news_date)
                if(date.year == 2011):
                    isBreak = True
                    break

                news_dict[idx] = {
                                'journal': "동아일보",
                                'date' : date.strftime("%Y-%m-%d"),
                                'title' : news_title,
                                'url' : news_url,
                                }
            except:
                news_dict[idx] = {
                                'journal': "동아일보",
                                'date' : "",
                                'title' : news_title,
                                'url' : news_url,
                                }
            idx += 1
        except NoSuchElementException as e:
            print(e)
            break

    if isBreak:
        break

    page += 20


browser.close()

In [ ]:
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '동아일보_2022-04-15_2012-01-01_{}_4.xlsx'.format(date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

news_df.head()

## newspaper 이용해서 기사 본문 가져오기

In [ ]:
from newspaper import Article
import os
import pandas as pd
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool

In [ ]:
df = pd.read_excel("/Users/yeonsun/Documents/Crawling/동아일보_2022-04-15_2012-01-01_2015-10-12 03:00:00.xlsx")
df.head()

In [ ]:
urls = df["url"].tolist()
news_list = list()

In [ ]:
def getText(url):
    article = Article(url)
    article.download()
    try:
        article.parse()
        news_list.append(
            {
                "journal": "동아일보",
                "date": article.publish_date.strftime("%Y-%m-%d"),
                "title": article.title,
                "url": url,
                "content": article.text,
            }
        )
        return ""
    except:
        news_list.append(
            {
                "journal": "동아일보",
                "date": "",
                "title": article.title,
                "url": url,
                "content": "",
            }
        )
        return ""
    
pool = ThreadPool(10)


In [ ]:
# open the urls in their own threads
# and return the results
for _ in tqdm(pool.imap_unordered(getText, urls), total=len(urls)):
    pass

# close the pool and wait for the work to finish
pool.close()
pool.join()

In [ ]:
col_name = ["journal",  "date", "title", "url", "content"]
news_df = pd.DataFrame(news_list, columns=col_name)

folder_path = os.getcwd()
csv_file_name = '동아일보_2022-04-15_2012-01-01_전체.csv'

news_df.to_csv(csv_file_name)

print("csv 저장 완료 | 경로 : {}\\{}\n".format(folder_path, csv_file_name))
news_df.head()

## connect to mongo and store the data in the mongo library

In [ ]:
from pymongo import MongoClient

#### local

In [ ]:
client = MongoClient(host='127.0.0.1', port=27017, username='BaekYeonsun', password='<password>')
db = client.database
collection = db.news
collection.insert_many(news_list)
print("push News Data in news DB")

#### cluster

In [ ]:
df = pd.read_excel("/Users/yeonsun/Documents/Crawling/동아일보_2022-04-15_2012-01-01_전체.xlsx")
df.head()

In [ ]:
news_dict = df.to_dict('records')

In [ ]:
client = MongoClient("mongodb+srv://BaekYeonsun:<password>@cluster.3dypr.mongodb.net/database?retryWrites=true&w=majority")
# for name in client.list_database_names():
#     print(name)
collection = client.database.news
try:
    collection.insert_many(news_dict)
except:
    pass
# for i in tqdm(range(0, len(news_dict))):
#     try:
#         collection.insert_one(news_dict[i])
#     except:
#         pass